# XADC

This example provides access to slow analog inputs on the E2 connector. Measurements are performed by the integrated Zynq XADC.

The XADC hardware is connected to the CPU over JTAG and the AXI4 bus. Each of this connections is supported by a IIO Linux kernel driver. The AXI4 driver provides more features. `libiio` is here used to interface with the AXI4 driver.

`libiio` should be updated to version 0.9 which provides native Python 3 bindings.
For now custumized bindings in the `iio.py` file are used.

In [ ]:
import iio

ctx = iio.Context()
dev = ctx.devices[3]

for chn in dev.channels:
    print ('{}: {} ({})'.format(chn.id, chn.name if chn.name else "", 'output' if chn.output else 'input'))
    raw    = chn.attrs['raw'].value
    scale  = chn.attrs['scale'].value
    if len(chn.attrs) == 6:
        offset = chn.attrs[b'offset'].value.decode()
        print ('\t({}+{})*{}/1000={}'.format(raw, offset, scale, (int(raw)+int(offset))*float(scale)/1000))
    else:
        print ('\t{}*{}/1000={}'.format(raw, scale, int(raw)*float(scale)/1000))

A simplple function can be used to combine the steps needed to get the input voltage on analog channels.

In [ ]:
channels = [dev.find_channel(name) for name in ('vaux0', 'vaux1', 'vaux8', 'vaux9')]
chn = len(channels)

def AIpinGetValue (channel):
    chn    = channels[channel]
    raw    = chn.attrs['raw'].value
    scale  = chn.attrs['scale'].value
    # resistor divider
    resdiv = 4.99 / (30.0 + 4.99)
    return (int(raw)*float(scale)/1000 / resdiv)

for i in range(chn):
    print("Measured voltage on AI[{}] = {} V".format(i, AIpinGetValue(i)))

## Measurement logging

This example is using a Pyhon list for the data sequence, which can become a performance issue for large data sets.

A better implementation would use a fixed `numpy` array for display data, and the rest of the data would be stored into a file.

In [ ]:
import time
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

In [ ]:
x = [0]
y = [[AIpinGetValue(ch)] for ch in range(chn)]

colors = ('red', 'blue', 'green', 'orange')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=400, plot_width=900, title="XADC log", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.y_range=Range1d(0, 5)
p.yaxis.axis_label='voltage [V]'
r = [p.line(x, y[ch], line_width=1, line_alpha=0.7, color=colors[ch], legend="AI "+str(ch)) for ch in range(chn)]

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

A sample is taken every `T` seconds.
This implementation suffers from time drift,
since the actual period is larger than `T`
by the time it takes to execute the rest of the loop.

In [ ]:
T = 1
#while True:
for i in range(200):
    x.append(x[-1]+T)
    for ch in range(chn):
        r[ch].data_source.data['y'].append(AIpinGetValue(ch))
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(T)